# Air Quality



In [6]:
# Packages
suppressMessages(library(tidyverse))
suppressMessages(library(lubridate))
suppressMessages(library(plyr))
suppressMessages(library(openxlsx))

suppressMessages(library(saqgetr))

In [7]:
# Working directory
setwd("~/Repositories/AirQualityCOVID")

# Source some scripts 


### Main Variables

In [8]:
site_type <- "traffic"
site_area <- "urban"

start_dt <- ymd_hms("2013-01-01 00:00:00")
end_dt <- ymd_hms("2020-12-30 00:00:00")

pollutants <- c("no", "no2", "o3", "pm10", "pm2.5")

## Available Station in Spain 

In [11]:
# AQ station in cities with more than 100000 inhabitants
sites.100mil <- read.xlsx("data/xlsx/estaciones-CA-JA.xlsx",
                          sheet="ciudades-100000-A") %>% 
                    select("Municipio", "Población",
                           "Estación.tráfico", "Código.estación") 

ERROR: Error: Can't subset columns that don't exist.
[31mx[39m Column `Poblaci<U+00F3>n` doesn't exist.


In [10]:
spain.sites <- get_saq_sites() %>%
    filter(country == "spain",
           site %in% sites.100mil$"Código.estación",
           site_type == "traffic",
           site_area == "urban",
           date_start <= start_dt,
           ) %>%
    select(site, site_name, latitude, longitude, elevation, 
           country, site_type, site_area, date_start, date_end)

ERROR: Error: Problem with `filter()` input `..2`.
[31mx[39m object 'sites.100mil' not found
[34mi[39m Input `..2` is `site %in% sites.100mil$"C<U+00F3>digo.estaci<U+00F3>n"`.


In [ ]:
sites.AQ <- merge(x = spain.sites,
                  y = sites.100mil,
                  by.x = "site", by.y="Código.estación",
                  all.x = TRUE) 

## Curation Estaciones Calidad del Aire

Estudiar de que contaminantes no hay datos suficientes para el estudio en cada estacion. Se ha de comprobar que hay datos ($> 80\%$) durante el periodo de interes (`1-Marzo-2020` <=> `1-Mayo-2020`). Tambien se comprueban si hay intervalos largos de tiempo sin datos, utilizando una resolucion minima semanal, mensual y anual.

|   site   |  Pollutant   |   start_yr   |    end_yr    | hv.min  | missing.wk | missing.mnth | missing.yr |
|----------|--------------|--------------|--------------|---------|------------|--------------|------------|
| es0001a  |     no2      |  01-01-2015  |  02-01-2015  |  TRUE   |     34     |      2       |     0      |
| es0001a  |     no       |  01-01-2015  |  02-01-2015  |  TRUE   |     40     |     12       |     1      |
| es0001a  |      o3      |  01-01-2015  |  02-01-2015  |  FALSE  |      4     |      0       |     0      |

Puesto que el estudio completo incluye 67 estaciones, supondria demasiado tiempo de computacion, por lo que en este notebook solo se trabajara con 3 estaciones. El calculo completo de todas las estaciones se realiza en un script de ```R```

In [ ]:
################################################################################

In [ ]:
hv.min.percent <- 0.8 # data > 80%
interest.prd <- c(ymd_hms("2020-03-01 00:00:00"),
                  ymd_hms("2020-05-01 00:00:00"))

sites.lv <- levels(as.factor(sitesAQ$site))[1:3]

In [ ]:
# Obtener que estaciones tienen los datos necesarios
curate.info <- do.call(rbind.fill,
                    lapply(sites.lv,
                           function(st, polluts){
                               do.call(rbind.fill,
                                       lapply(polluts,
                                              info_sitesAQ,
                                              st, start_dt, end_dt))
                           }, pollutants))

| Parameter | Value |
|----|----|
| hv.min | TRUE |
|miss.yr | $< 5$ |

In [ ]:
valid.info <- curate.info[curate.info$hv.min == TRUE,]
valid.info <- valid.info[valid.info$miss.yr < 5,]

checked_sitesAQ <- merge(x = valid.info %>%
                             select(site, Pollutant),
                         y = sitesAQ,
                         by = "site", all.x = T, all.y=F) 

### Guardar Datos en csv <a id="saveAQ"></a>

```R
write.csv(valid.info, 
          "data/Curation/AirQuality/checked_sitesAQ.csv", row.names=FALSE)
```